In [29]:
# from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import pandas as pd

hunt = pd.read_csv('Data\\Hunt\\huntfile.tsv', delimiter='\t')
khar = pd.read_csv('Data\\Kharchenko\\kharchenkofile.tsv', delimiter='\t')
dias = pd.read_csv('Data\\Dias\\diasfile.tsv', delimiter='\t')
xmatch = pd.read_csv('Data\\Hunt\\xmatchfile.tsv', delimiter='\t')
kharchenko_xm = xmatch.query('SourceCat == "Kharchenko+13"', )

khar
# def datahandler(filename):
#     authorname = filename.split('\\')
#     hdul = fits.open(filename)
#     hdr1 = hdul[0].header #first and second header
#     hdr2 = hdul[1].header
#     data, cols = hdul[1].data, hdul[1].columns
#     metadata = [ [ hdr2[f"TTYPE{i+1}"], hdr2.comments[f"TTYPE{i+1}"] ] for i, j in enumerate(cols.names) ]
#     # mydict[authorname]['hdul'] = hdul
#     return hdul[1].data, hdul[1].columns, metadata


FileNotFoundError: [Errno 2] No such file or directory: 'Data\\Dias\\diasfile.tsv'

## Comparing catalogues

The goal now is to get the correct clusters from the Kharchenko, Dias and CanatGaudin catalogues with respect to the Hunt paper. Pretty much recreate the table 4 from the hunt paper. We want to cross match the catalogues. We first want to find the $n_\mathrm{clusters}$ for each catalogue.

NameError: name 'pd' is not defined

In [73]:
#getting ocs from kharchenko
mask_khar = (data_khar['Type']!='a') & (data_khar['Type'] != 'm') & (data_khar['Type'] != 's')
khar_ocs = data_khar[mask_khar]

print(f"The amount of clusters in the Kharchenko catalogue is indeed {len(khar_ocs)}")

#significant cluster from hunt 
# mask_hunt = data_hunt['CST'] >5
# print(f"Amount of significant cluster is {len(data_hunt[mask_hunt])}")

The amount of clusters in the Kharchenko catalogue is indeed 2935


In [56]:
all_names = np.concatenate([name.split(',') for name in data_hunt['AllNames']]) #get all names from hunt file

cross = np.in1d(list(khar_ocs['Name']), all_names) #intersect hunt all names and kharchenko files
np.unique(khar_ocs[cross]['Name'])

print(f"The amount to found Kharchenko OCs are {len(np.unique(khar_ocs[cross]['Name']))} != 1513 as in the paper")

The amount to found Kharchenko OCs are 1538 != 1513 as in the paper
